In [45]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import pandas as pd

#### Load Data

In [ ]:
speeches = pd.read_csv("../python/")

In [42]:
model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)


In [40]:
sentences = [['drink','not','good'],
			['felt','superb'],
			['just','good','ambience'],
			['bad','taste'],
			['parking','problem'],
            ['fantastic','food']]

model.train(sentences, total_examples=1, epochs=1)

(1, 14)

In [43]:
nottrained = model.wv['good']

In [41]:
trained = model.wv['good']

In [44]:
trained == nottrained

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])